In [ ]:
!pip install pyswarm

In [ ]:
# from collections import Iterable, OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from collections.abc import Iterable
from collections import OrderedDict
from google.colab import drive
from torch.nn import Parameter
from datetime import datetime
from functools import partial
import torch.optim as optim
from pyswarm import pso
import torch.nn as nn
from math import sqrt
import pandas as pd
import numpy as np
import inspect
import torch
import time

# Load data

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wc_real/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'GKYZV_data.csv'
GKYZV_data = pd.read_csv(path + filename)

In [ ]:
GKYZV_data.columns

Index(['Date', 'GKYZV', 'log_returns', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', 'vix', 'interest_rate', 'inflation_rate',
       'usd_eur', 'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn',
       'gt_data', 'unemployment_rate'],
      dtype='object')

In [ ]:
GKYZV_data.drop(columns=['vix', 'interest_rate', 'inflation_rate', 'usd_eur', 'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data', 'unemployment_rate'], inplace=True)

In [ ]:
GKYZV_data

,Date,GKYZV,log_returns,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)"
0,2000-02-02,-0.001436,0.000077,-0.001864,-0.001725,-0.001864,-0.001725,-0.001971,-0.001822,-0.001782,-0.001782,-0.001782,-0.001782
1,2000-02-03,-0.001275,0.000020,-0.001554,-0.001557,-0.001554,-0.001557,-0.001745,-0.001612,-0.001519,-0.001519,-0.001519,-0.001519
2,2000-02-04,-0.000941,0.000152,-0.001080,-0.001163,-0.001080,-0.001163,-0.001302,-0.001217,-0.001076,-0.001076,-0.001076,-0.001076
3,2000-02-07,-0.000613,0.000090,-0.000627,-0.000738,-0.000627,-0.000738,-0.000863,-0.000784,-0.000678,-0.000678,-0.000678,-0.000678
4,2000-02-08,-0.000297,-0.000335,-0.000202,-0.000335,-0.000202,-0.000335,-0.000364,-0.000368,-0.000311,-0.000311,-0.000311,-0.000311
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6179,2024-08-26,0.000421,-0.000161,-0.000358,-0.000493,-0.000358,-0.000493,-0.001023,-0.000502,-0.000690,-0.000690,-0.000690,-0.000690
6180,2024-08-27,0.000042,-0.000193,-0.000694,-0.000707,-0.000694,-0.000707,-0.001244,-0.000717,-0.000899,-0.000899,-0.000899,-0.000899
6181,2024-08-28,-0.000355,-0.000154,-0.000954,-0.000846,-0.000954,-0.000846,-0.001311,-0.000852,-0.001012,-0.001012,-0.001012,-0.001012
6182,2024-08-29,-0.000779,-0.000150,-0.001147,-0.000937,-0.001147,-0.000937,-0.001324,-0.000933,-0.001067,-0.001067,-0.001067,-0.001067


In [ ]:
set_index(GKYZV_data)

In [ ]:
tm_data = GKYZV_data

In [ ]:
GKYZV_data = GKYZV_data.drop(columns=['GKYZV'])

In [ ]:
GKYZV_data['GKYZV'] = tm_data['GKYZV']

In [ ]:
GKYZV_data

,log_returns,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",GKYZV
Date,,,,,,,,,,,,
2000-02-02,0.000077,-0.001864,-0.001725,-0.001864,-0.001725,-0.001971,-0.001822,-0.001782,-0.001782,-0.001782,-0.001782,-0.001436
2000-02-03,0.000020,-0.001554,-0.001557,-0.001554,-0.001557,-0.001745,-0.001612,-0.001519,-0.001519,-0.001519,-0.001519,-0.001275
2000-02-04,0.000152,-0.001080,-0.001163,-0.001080,-0.001163,-0.001302,-0.001217,-0.001076,-0.001076,-0.001076,-0.001076,-0.000941
2000-02-07,0.000090,-0.000627,-0.000738,-0.000627,-0.000738,-0.000863,-0.000784,-0.000678,-0.000678,-0.000678,-0.000678,-0.000613
2000-02-08,-0.000335,-0.000202,-0.000335,-0.000202,-0.000335,-0.000364,-0.000368,-0.000311,-0.000311,-0.000311,-0.000311,-0.000297
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,-0.000161,-0.000358,-0.000493,-0.000358,-0.000493,-0.001023,-0.000502,-0.000690,-0.000690,-0.000690,-0.000690,0.000421
2024-08-27,-0.000193,-0.000694,-0.000707,-0.000694,-0.000707,-0.001244,-0.000717,-0.000899,-0.000899,-0.000899,-0.000899,0.000042
2024-08-28,-0.000154,-0.000954,-0.000846,-0.000954,-0.000846,-0.001311,-0.000852,-0.001012,-0.001012,-0.001012,-0.001012,-0.000355


# Split train, validation and test data

In [ ]:
# train_end = datetime(2020, 6, 30)
# test_start = datetime(2020, 10, 1)
# val_size = 0.1

In [ ]:
# target = ['GKYZV', 'CCV', 'PV', 'GKV', 'RSV', 'YZV', 'GKYZV']
# test_data = data.loc[test_start:]

# train_val = data.loc[:train_end]
# train_data, val_data = train_test_split(train_val, test_size=val_size, shuffle=False)

# X_train = train_data.drop(columns=target)
# y_train = train_data[target]

# X_val = val_data.drop(columns=target)
# y_val = val_data[target]

# X_test = test_data.drop(columns=target)
# y_test = test_data[target]

# X_train = np.array(X_train)
# y_train = np.array(y_train)

# X_val = np.array(X_val)
# y_val = np.array(y_val)

# X_test = np.array(X_test)
# y_test = np.array(y_test)

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  X_train = train_data.drop(columns=[target])
  y_train = train_data[target]

  X_val = val_data.drop(columns=[target])
  y_val = val_data[target]

  X_test = test_data.drop(columns=[target])
  y_test = test_data[target]

  return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(GKYZV_data, 'GKYZV')

# Model

In [ ]:
class BDropout(torch.nn.Dropout):

    """Binary dropout with regularization and resampling.

    See: Gal Y., Ghahramani Z., "Dropout as a Bayesian Approximation:
    Representing Model Uncertainty in Deep Learning", 2016.
    """

    # def __init__(self, rate=0.1, reg=1.0, **kwargs):
    #     """Constructs a BDropout.

    #     Args:
    #         rate (float): Dropout probability.
    #         reg (float): Regularization scale.
    #     """
    #     super(BDropout, self).__init__(**kwargs)
    #     self.register_buffer("rate", torch.tensor(rate))
    #     self.p = 1 - self.rate
    #     self.register_buffer("reg", torch.tensor(reg))
    #     self.register_buffer("noise", torch.bernoulli(self.p))


    def __init__(self, rate=0.1, reg=1.0, **kwargs):
        """Constructs a BDropout.

        Args:
            rate (float): Dropout probability.
            reg (float): Regularization scale.
        """
        super(BDropout, self).__init__(p=rate, **kwargs)
        self.register_buffer("rate", torch.tensor(rate))
        self.p = 1 - self.rate
        self.register_buffer("reg", torch.tensor(reg))
        self.register_buffer("noise", torch.bernoulli(self.p))

    def regularization(self, weight, bias):
        """Computes the regularization cost.

        Args:
            weight (Tensor): Weight tensor.
            bias (Tensor): Bias tensor.

        Returns:
            Regularization cost (Tensor).
        """
        self.p = 1 - self.rate
        weight_reg = self.p * (weight**2).sum()
        bias_reg = (bias**2).sum() if bias is not None else 0
        return self.reg * (weight_reg + bias_reg)

    def resample(self):
        """Resamples the dropout noise."""
        self._update_noise(self.noise)

    def _update_noise(self, x):
        """Updates the dropout noise.

        Args:
            x (Tensor): Input.
        """
        self.p = 1 - self.rate
        self.noise.data = torch.bernoulli(self.p.expand(x.shape))

    def forward(self, x, resample=False, mask_dims=0, **kwargs):
        """Computes the binary dropout.

        Args:
            x (Tensor): Input.
            resample (bool): Whether to force resample.
            mask_dims (int): Number of dimensions to sample noise for
                (0 for all).

        Returns:
            Output (Tensor).
        """
        sample_shape = x.shape[-mask_dims:]
        if sample_shape != self.noise.shape:
            sample = x.view(-1, *sample_shape)[0]
            self._update_noise(sample)
        elif resample:
            return x * torch.bernoulli(self.p.expand(x.shape))

        return x * self.noise

    def extra_repr(self):
        """Formats module representation.

        Returns:
            Module representation (str).
        """
        return "rate={}".format(self.rate)

In [ ]:
class CDropout(BDropout):

    """Concrete dropout with regularization and resampling.

    See: Gal Y., Hron, J., Kendall, A. "Concrete Dropout", 2017.
    """

    # def __init__(self, temperature=0.1, rate=0.5, reg=1.0, **kwargs):
    #     """Constructs a CDropout.

    #     Args:
    #         temperature (float): Temperature.
    #         rate (float): Initial dropout rate.
    #         reg (float): Regularization scale.
    #     """
    #     super(CDropout, self).__init__(rate, reg, **kwargs)
    #     self.temperature = Parameter(
    #         torch.tensor(temperature), requires_grad=False)

    #     # We need to constrain p to [0, 1], so we train logit(p).
    #     self.logit_p = Parameter(-torch.log(self.p.reciprocal() - 1.0))

    def __init__(self, temperature=0.1, rate=0.5, reg=1.0, **kwargs):
        """Constructs a CDropout.

        Args:
            temperature (float): Temperature.
            rate (float): Initial dropout rate.
            reg (float): Regularization scale.
        """
        super(CDropout, self).__init__(rate, reg, **kwargs)
        self.temperature = torch.nn.Parameter(
            torch.tensor(temperature), requires_grad=False)

        # We need to constrain p to [0, 1], so we train logit(p).
        self.logit_p = torch.nn.Parameter(-torch.log(self.p.reciprocal() - 1.0))

    def regularization(self, weight, bias):
        """Computes the regularization cost.

        Args:
            weight (Tensor): Weight tensor.
            bias (Tensor): Bias tensor.

        Returns:
            Regularization cost (Tensor).
        """
        self.p.data = self.logit_p.sigmoid()
        reg = super(CDropout, self).regularization(weight, bias)
        reg -= -(1 - self.p) * (1 - self.p).log() - self.p * self.p.log()
        return reg

    def _update_noise(self, x):
        """Updates the dropout noise.

        Args:
            x (Tensor): Input.
        """
        self.noise.data = torch.rand_like(x)

    def forward(self, x, resample=False, mask_dims=0, **kwargs):
        """Computes the concrete dropout.

        Args:
            x (Tensor): Input.
            resample (bool): Whether to force resample.
            mask_dims (int): Number of dimensions to sample noise for
                (0 for all).

        Returns:
            Output (Tensor).
        """
        sample_shape = x.shape[-mask_dims:]
        noise = self.noise
        if sample_shape != noise.shape:
            sample = x.view(-1, *sample_shape)[0]
            self._update_noise(sample)
            noise = self.noise
        elif resample:
            noise = torch.rand_like(x)

        self.p.data = self.logit_p.sigmoid()
        concrete_p = self.logit_p + noise.log() - (1 - noise).log()
        concrete_noise = (concrete_p / self.temperature).sigmoid()

        return x * concrete_noise

    def extra_repr(self):
        """Formats module representation.

        Returns:
            Module representation (str).
        """
        return "temperature={}".format(self.temperature)

In [ ]:
class BSequential(torch.nn.Sequential):

    """Extension of Sequential module with regularization and resampling."""

    def resample(self):
        """Resample all child modules."""
        for child in self.children():
            if isinstance(child, BDropout):
                child.resample()

    def regularization(self):
        """Computes the total regularization cost of all child modules.

        Returns:
            Total regularization cost (Tensor).
        """
        reg = torch.tensor(0.0)
        children = list(self._modules.values())
        for i, child in enumerate(children):
            if isinstance(child, BSequential):
                reg += child.regularization()
            elif isinstance(child, BDropout):
                for next_child in children[i:]:
                    if hasattr(next_child, "weight") and hasattr(
                            next_child, "bias"):
                        reg += child.regularization(next_child.weight,
                                                    next_child.bias)
                        break
        return reg

    def forward(self, x, resample=False, **kwargs):
        """Computes the model.

        Args:
            x (Tensor): Input.
            resample (bool): Whether to force resample.

        Returns:
            Output (Tensor).
        """
        for module in self._modules.values():
            if isinstance(module, (BDropout, BSequential)):
                x = module(x, resample=resample, **kwargs)
            else:
                x = module(x)
        return x

In [ ]:
def bayesian_model(in_features,
                   out_features,
                   hidden_features,
                   nonlin=nn.ReLU,
                   output_nonlin=None,
                   weight_initializer=partial(
                       nn.init.xavier_normal_,
                       gain=nn.init.calculate_gain("relu")),
                   bias_initializer=partial(
                       nn.init.uniform_, a=-1.0, b=1.0),
                   dropout_layers=CDropout,
                   input_dropout=None):
    """Constructs and initializes a Bayesian neural network with dropout."""
    dims = [in_features] + hidden_features
    if not isinstance(dropout_layers, Iterable):
        dropout_layers = [dropout_layers] * len(hidden_features)

    modules = OrderedDict()

    if inspect.isclass(input_dropout):
        input_dropout = input_dropout()
    if input_dropout is not None:
        modules["drop_in"] = input_dropout

    for i, (din, dout) in enumerate(zip(dims[:-1], dims[1:])):
        drop_i = dropout_layers[i]
        if inspect.isclass(drop_i):
            drop_i = drop_i()

        modules["fc_{}".format(i)] = nn.Linear(din, dout)
        if drop_i is not None:
            modules["drop_{}".format(i)] = drop_i
        modules["nonlin_{}".format(i)] = nonlin()

    modules["fc_out"] = nn.Linear(dims[-1], out_features)
    if output_nonlin is not None:
        modules["nonlin_out"] = output_nonlin()

    def init(module):
        if callable(weight_initializer) and hasattr(module, "weight"):
            weight_initializer(module.weight)
        if callable(bias_initializer) and hasattr(module, "bias"):
            if module.bias is not None:
                bias_initializer(module.bias)

    net = BSequential(modules)
    net.apply(init)

    return net

In [ ]:
# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Early stopping implementation
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_val_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_val_loss is None:
            self.best_val_loss = val_loss
        elif val_loss < self.best_val_loss - self.min_delta:
            self.best_val_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# Define parameter bounds
def parameter_bounds():
    # Number of layers: 1 to 5
    num_layers_bounds = (1, 5)

    # Number of neurons per layer: 10 to 200
    num_neurons_bounds = (10, 200)

    # Dropout rate: 0 to 0.5
    dropout_rate_bounds = (0.0, 0.5)

    # Learning rate: 0.0001 to 0.01
    learning_rate_bounds = (0.0001, 0.01)

    # Number of epochs: 50 to 500
    num_epochs_bounds = (150, 500)

    return num_layers_bounds, num_neurons_bounds, dropout_rate_bounds, learning_rate_bounds, num_epochs_bounds

In [ ]:
def objective_function(params):
    # Unpack parameters from PSO
    num_layers, num_neurons, dropout_rate, learning_rate, num_epochs = params

    # Define the model with the given parameters
    model = bayesian_model(
        in_features=X_train.shape[1],
        out_features=1,
        hidden_features=[int(num_neurons)] * int(num_layers),
        nonlin=nn.ReLU,
        output_nonlin=None,
        dropout_layers=CDropout,
        input_dropout=CDropout(temperature=0.1, rate=dropout_rate, reg=1.0)
    )

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=10, min_delta=0.01)

    # Track the best validation loss
    best_val_loss = float('inf')

    # Training loop with the number of epochs specified in the params
    for epoch in range(int(num_epochs)):
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation phase (after each epoch)
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)

        # Check and update the best validation loss
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()

        # Early stopping condition
        early_stopping(val_loss.item())
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    # Return the best validation loss for PSO optimization
    return best_val_loss

In [ ]:
# Define the bounds for each parameter
def get_bounds():
    num_layers_bounds, num_neurons_bounds, dropout_rate_bounds, learning_rate_bounds, num_epochs_bounds = parameter_bounds()
    return [
        [num_layers_bounds[0], num_neurons_bounds[0], dropout_rate_bounds[0], learning_rate_bounds[0], num_epochs_bounds[0]],
        [num_layers_bounds[1], num_neurons_bounds[1], dropout_rate_bounds[1], learning_rate_bounds[1], num_epochs_bounds[1]]
    ]

In [ ]:
bounds = get_bounds()

In [ ]:
lower_bounds = bounds[0]
upper_bounds = bounds[1]

swarmsize = 20
maxiter = 50

In [ ]:
start_time = time.time()

best_params, best_val = pso(objective_function, lb=lower_bounds, ub=upper_bounds, swarmsize=swarmsize, maxiter=maxiter, debug=True)

end_time = time.time()
execution_time = end_time - start_time

No constraints given.


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5055])) that is different to the input size (torch.Size([5055, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([562])) that is different to the input size (torch.Size([562, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Early stopping triggered at epoch 68
Early stopping triggered at epoch 18
Early stopping triggered at epoch 27
Early stopping triggered at epoch 47
Early stopping triggered at epoch 45
Early stopping triggered at epoch 36
Early stopping triggered at epoch 40
Early stopping triggered at epoch 39
Early stopping triggered at epoch 49
Early stopping triggered at epoch 34
Early stopping triggered at epoch 14
Early stopping triggered at epoch 50
Early stopping triggered at epoch 30
Early stopping triggered at epoch 43
Early stopping triggered at epoch 48
Early stopping triggered at epoch 36
Early stopping triggered at epoch 46
Early stopping triggered at epoch 31
Early stopping triggered at epoch 27
Early stopping triggered at epoch 73
Early stopping triggered at epoch 53
Early stopping triggered at epoch 36
Early stopping triggered at epoch 59
Early stopping triggered at epoch 47
Early stopping triggered at epoch 52
Early stopping triggered at epoch 48
Early stopping triggered at epoch 42
E

In [ ]:
execution_time

11032.199305534363

In [ ]:
best_params

array([3.45883532e+00, 9.50375308e+01, 2.67458605e-01, 9.58336464e-03,
       2.17120909e+02])

In [81]:
# Function to train and evaluate the model with given hyperparameters
def train_and_evaluate_model(params):
    n_layers, n_neurons, drop_rate, learning_rate, epochs = params

    # Define the model
    hidden_features = [int(n_neurons)] * int(n_layers)
    model = bayesian_model(
        in_features=X_train.shape[1],
        out_features=1,
        hidden_features=hidden_features,
        dropout_layers=CDropout(rate=drop_rate),  # Dropout layer without partial
    )

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Early stopping
    early_stopping = EarlyStopping(patience=10, min_delta=1e-4)

    # Training loop
    for epoch in range(int(epochs)):
        model.train()
        optimizer.zero_grad()

        output = model(X_train)
        loss = criterion(output, y_train)

        loss.backward()
        optimizer.step()

        model.eval()
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)

        print(f'Epoch {epoch+1}/{int(epochs)}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')

        early_stopping(val_loss.item())
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    # Test the model
    model.eval()
    test_output = model(X_test)
    mse = criterion(test_output, y_test)
    rmse = mse ** 0.5
    mae = nn.L1Loss()(y_test, test_output).item()
    mape = torch.mean(torch.abs((y_test - test_output) / y_test)) * 100

    print(f'Test Loss: {mse.item()}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape.item()}%')

    return mse, rmse, mae, mape, test_output

In [82]:
# best_params = [3.62253629e+00, 1.12321717e+02, 2.77934900e-01, 5.49072091e-03, 2.22486413e+02]

In [83]:
mse, rmse, mae, mape, test_output = train_and_evaluate_model(best_params)

Epoch 1/217, Loss: 0.4480665326118469, Val Loss: 2.4375600814819336
Epoch 2/217, Loss: 2.4921984672546387, Val Loss: 0.4732680022716522
Epoch 3/217, Loss: 0.5126733183860779, Val Loss: 0.3239002525806427
Epoch 4/217, Loss: 0.2966645658016205, Val Loss: 0.6854788064956665
Epoch 5/217, Loss: 0.7038601040840149, Val Loss: 0.6899814605712891
Epoch 6/217, Loss: 0.687865674495697, Val Loss: 0.38812363147735596
Epoch 7/217, Loss: 0.3802584111690521, Val Loss: 0.16585767269134521
Epoch 8/217, Loss: 0.16018086671829224, Val Loss: 0.09008194506168365
Epoch 9/217, Loss: 0.09846585988998413, Val Loss: 0.13995704054832458
Epoch 10/217, Loss: 0.14005962014198303, Val Loss: 0.18974857032299042
Epoch 11/217, Loss: 0.18526731431484222, Val Loss: 0.2088460624217987
Epoch 12/217, Loss: 0.1935596764087677, Val Loss: 0.15627245604991913
Epoch 13/217, Loss: 0.16813185811042786, Val Loss: 0.12113477289676666
Epoch 14/217, Loss: 0.13178357481956482, Val Loss: 0.1019698977470398
Epoch 15/217, Loss: 0.096247553

In [84]:
print(f'Final Test Metrics -> MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape.item()}%')

Final Test Metrics -> MSE: 0.0027107272762805223, RMSE: 0.052064646035432816, MAE: 0.03021499142050743, MAPE: 899931.0625%


In [85]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/GKYZV/'

# df_pred = pd.DataFrame(test_output.detach().numpy().flatten(), columns=['bnn_pred'])
# filename = 'BNN_pred.csv'
# df_pred.to_csv(path_1 + filename)